In [14]:
import pandas as pd
import numpy as np
import sys

In [15]:
# Getting stocks score from 2019-01-01 to 2021-03-01
import yfinance as yf
stocks_topull = ['GME','TSLA','SPY','^IXIC','DJIA']
start_date = '2019-01-01'
end_date = '2021-03-01'
 
#input string variable
def getting_spy_score(start,end,ticker):
    pull_data = yf.Ticker(ticker)
    stock_data = pull_data.history(start = start, end = end)
    stock_data = stock_data['Close'] #taking Close score as daily score
    return stock_data
 
for stock_no, stock_i in enumerate(stocks_topull):
    sample_stock = getting_spy_score(start_date,end_date,stock_i)
    sample_stock = pd.DataFrame(sample_stock).reset_index()
    sample_stock.columns = ['Date_Type',stock_i]
 
    if stock_no == 0:
        full_stock_df = sample_stock.copy()
    else:
        full_stock_df = pd.merge(full_stock_df, sample_stock,
                                 on=['Date_Type'],
                                 how='outer')
        
full_stock_df['Date'] = full_stock_df['Date_Type'].astype(str)

In [22]:
## Import Sentiment Data
sentiment_data = pd.read_csv("Sentiment_Data.csv")

In [23]:
## Combine with Stocks data
sentiment_stocks = pd.merge(sentiment_data,full_stock_df,
                      on = ['Date'],
                      how = 'left'
                     )

In [24]:
sentiment_stocks = sentiment_stocks[['Date', 'score', 'sentiment_score', 'bull_scores', 'bear_scores',
       'num_comments', 'num_crossposts', 'GME', 'TSLA', 'SPY',
       '^IXIC', 'DJIA']].copy()
sentiment_stocks.columns = ['date', 'score', 'sentiment_score', 'bull_scores', 'bear_scores',
       'num_comments', 'num_crossposts', 'GME_y', 'TSLA_y', 'SPY_y','^IXIC_y', 'DJIA_y']

In [25]:
## Processing Data
full_final = sentiment_stocks.copy()
stock_cols = [i+"_y" for i in full_stock_df.columns if 'Date' not in i]
rows_drop = []
for row_i in range(len(full_final)):
    if row_i == 0:
        if any(np.isnan(full_final[stock_i][row_i]) for stock_i in stock_cols):
            rows_drop += [row_i]
    elif row_i == (len(full_final)-1):
        if any(np.isnan(full_final[stock_i][row_i]) for stock_i in stock_cols):
            rows_drop += [row_i]
    else:
        for stock_i in stock_cols:
            
            sys.stdout.write("\rRunning for {}/{} ({})         ".format(
                row_i+1, len(full_final),
                stock_i
            ))
            sys.stdout.flush()
            
            if np.isnan(full_final[stock_i][row_i]):
                before_i = 1
                before_status_done = True
                while before_status_done:
                    
                    if (row_i-before_i) < 0:
                        before_status_done = False
                    elif np.isnan(full_final[stock_i][row_i-before_i]):
                        before_i += 1
                    else:
                        before_value = full_final[stock_i][row_i-before_i]
                        before_status_done = False

                after_i = 1
                after_status_done = True
                while after_status_done:
                    if (row_i+after_i) >= len(full_final):
                        after_status_done = False
                    elif np.isnan(full_final[stock_i][row_i+after_i]):
                        after_i += 1
                    else:
                        after_value = full_final[stock_i][row_i+after_i]
                        after_status_done = False
                
                if (row_i-before_i) < 0 or (row_i+after_i) >= len(full_final):
                    rows_drop += [row_i]
                else:
                    full_final[stock_i][row_i] = np.mean([after_value, before_value])
                
            else:
                continue

Running for 20/787 (GME_y)           

C:\Users\Ace\Anaconda2\envs\py37\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running for 786/787 (DJIA_y)          

In [26]:
sentiment_stocks = full_final.copy()

In [31]:
## Import network_data
network_data = pd.read_csv("network_metrics_daily.csv")

In [32]:
## Processing network data
full_final = network_data.copy()
stock_cols = [i for i in full_final.columns if 'date' not in i]

rows_drop = []
for row_i in range(len(full_final)):
    if row_i == 0:
        if any(np.isnan(full_final[stock_i][row_i]) for stock_i in stock_cols):
            rows_drop += [row_i]
    elif row_i == (len(full_final)-1):
        if any(np.isnan(full_final[stock_i][row_i]) for stock_i in stock_cols):
            rows_drop += [row_i]
    else:
        for stock_i in stock_cols:
            
            sys.stdout.write("\rRunning for {}/{} ({})                ".format(
                row_i+1, len(full_final),
                stock_i
            ))
            sys.stdout.flush()
            
            if np.isnan(full_final[stock_i][row_i]):
                before_i = 1
                before_status_done = True
                while before_status_done:
                    
                    if (row_i-before_i) < 0:
                        before_status_done = False
                    elif np.isnan(full_final[stock_i][row_i-before_i]):
                        before_i += 1
                    else:
                        before_value = full_final[stock_i][row_i-before_i]
                        before_status_done = False

                after_i = 1
                after_status_done = True
                while after_status_done:
                    if (row_i+after_i) >= len(full_final):
                        after_status_done = False
                    elif np.isnan(full_final[stock_i][row_i+after_i]):
                        after_i += 1
                    else:
                        after_value = full_final[stock_i][row_i+after_i]
                        after_status_done = False
                
                if (row_i-before_i) < 0 or (row_i+after_i) >= len(full_final):
                    rows_drop += [row_i]
                else:
                    full_final[stock_i][row_i] = np.mean([after_value, before_value])
                
            else:
                continue

Running for 354/790 (investing_author_cnt)                            

C:\Users\Ace\Anaconda2\envs\py37\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running for 789/790 (connectivity)                                    

In [33]:
full_final = full_final.drop(list(set(rows_drop))).copy()

In [34]:
## combine with sentiment analysis
full_all = pd.merge(sentiment_stocks, full_final,
                   on=['date'],
                   how='inner')

In [39]:
full_all.to_csv("Full_Dataset.csv",index=False)